ДЗ № 5 Тема "POS-tagger и NER"
---

**Задание 1**. Написать теггер на данных с русским языком
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы
 
**Задание 2**. Проверить насколько хорошо работает NER

*данные брать из http://www.labinform.ru/pub/named_entities/*

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы
 1. передаём в сетку токен и его соседей
 2. передаём в сетку только токен
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется


https://github.com/guillaumegenthial/tf_ner - Simple and Efficient Tensorflow implementations of NER models with tf.estimator and tf.data

# Задание 1

## 1. Проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации

In [16]:
!pip install pyconll

In [28]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

### Скачиваем данные

In [3]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-08-08 11:27:53--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train.conllu’

ru_syntagrus-ud-tra 100%[===================>]  77.28M   205MB/s    in 0.4s    

2021-08-08 11:27:56 (205 MB/s) - ‘ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-08-08 11:27:56--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.1

Загрузка данных

In [4]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [5]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [6]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [7]:
# Заполним данными
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

### Проверим UnigramTagger, BigramTagger, TrigramTagger и их комбинации

In [8]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 3)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 3)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 3)},\n')

Accuracy:
Unigram Tagger: 0.877,
Bigram Tagger: 0.696,
Trigram Tagger: 0.248,
Bigram and Unigram Tagger: 0.883,
Trigram, Bigram and Unigram Tagger: 0.882,



## 2. Написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [10]:
# Переведём тренировочный датасет в списки слов и списки POS-разметки
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [11]:
train_tok[:10], train_label[:10]

(['Анкета',
  '.',
  'Начальник',
  'областного',
  'управления',
  'связи',
  'Семен',
  'Еремеевич',
  'был',
  'человек'],
 ['NOUN',
  'PUNCT',
  'NOUN',
  'ADJ',
  'NOUN',
  'NOUN',
  'PROPN',
  'PROPN',
  'AUX',
  'NOUN'])

In [12]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
train_enc_labels

array([ 7, 13,  7, ...,  9,  7, 13])

In [13]:
test_enc_labels = le.transform(test_label)
test_enc_labels

array([ 7, 13,  1, ..., 10, 16, 13])

In [14]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [24]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.9437872813668992
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.947140498095912
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9487749806221144


# Задание 2. Проверить насколько хорошо работает NER

*данные брать из http://www.labinform.ru/pub/named_entities/*

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы
 1. передаём в сетку токен и его соседей
 2. передаём в сетку только токен
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

In [5]:
!pip install corus

     |████████████████████████████████| 83 kB 1.9 MB/s 


In [6]:
import corus
from corus import load_ne5
import nltk

In [7]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [8]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-08-08 13:08:07--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   371KB/s    in 5.7s    

2021-08-08 13:08:13 (323 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [9]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [10]:
records = load_ne5('Collection5/')
records

<generator object load_ne5 at 0x7fd1cffff750>

In [11]:
document = next(records).text
document

'Полиция ЮАР применила свето-шумовые гранаты на акции против Обамы\r\n\r\n\r\nДемонстранты выступали против приезда президента США Барака Обамы в ЮАР. В момент столкновений Обама с супругой встречались с родными Манделы.\r\n Столкновения полиции с демонстрантами, выступающими против приезда президента США Барака Обамы в ЮАР, произошли в субботу на окраине Йоханнесбурга, передают западные информагентства.\r\n\r\nПолиция ЮАР светошумовыми гранатами разогнала противников приезда Обамы\r\n\r\nПо сообщениям, вооруженные полицейские применили для разгона толпы свето-шумовые гранаты. Стычки с полицией произошли в поселении Соуэто, находящемся на юго-западной окраине города у здания кампуса университета Йоханнесбурга, где в субботу Обама встречается с молодежью.\r\n\r\nСтолкновения произошли в то время, когда Обама и его супруга Мишель встречались с родными Нельсона Манделы в здании фонда экс-президента ЮАР, отмечает агентство Рейтер.\r\n\r\nВ начале июня Мандела попал в госпиталь в Претории с

In [12]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Барака Обамы', 'PERSON'),
 ('Джейкоб Зума', 'PERSON'),
 ('Мандела', 'PERSON'),
 ('Манделы', 'PERSON'),
 ('Мишель', 'PERSON'),
 ('Полиция', 'PERSON'),
 ('ЮАР', 'ORGANIZATION')}

**Вывод**: Николай Диденко и Игорь волков правильно распознаны как персона и ЗАТО - как организация. А вот Дума Северска, новости, новый, Северск и Томской - ошибочно помечены как персона.

Качество весьма посредственное.

## Проверить deeppavlov

In [35]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

Found existing installation: tensorflow 2.5.0
Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 241 kB 7.9 MB/s 
     |████████████████████████████████| 2.6 MB 52.3 MB/s 
     |████████████████████████████████| 3.3 MB 41.7 MB/s 
     |████████████████████████████████| 895 kB 47.8 MB/s 
     |████████████████████████████████| 636 kB 57.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 901 kB 7.8 MB/s 
     |████████████████████████████████| 2.9 MB 27.9 MB/s 
     |████████████████████████████████| 6.7 MB 21.2 MB/s 
     |████████████████████████████████| 65 kB 4.1 MB/s 
     |████████████████████████████████| 7.3 MB 40.0 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 859 kB 42.7 MB/s 
     |████████████████████████████████| 

In [36]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

2021-08-08 12:27:20.35 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
     |████████████████████████████████| 110.5 MB 1.7 kB/s 
     |████████████████████████████████| 3.8 MB 47.4 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
     |████████████████████████████████| 503 kB 37.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=c28f8b73f44b64e8608f9b1548e400dee84ac9951643167fed7226569c6eb9ef
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Un

In [37]:
import deeppavlov
from deeppavlov import configs, build_model

In [42]:
document

'\r\n\r\nСеверск до конца года останется без сити-менеджера\r\n\r\n\r\nДума Северска признала несостоявшимся конкурс на пост главы администрации. Новый конкурс, скорее всего, будет объявлен осенью.\r\n Дума самого большого закрытого города системы Росатома — города Северск в Томской области — в четверг признала несостоявшимся конкурс на пост главы администрации, так как выдвинулся всего один претендент, сообщил РИА Новости глава конкурсной комиссии, депутат Константин Изместьев.\r\n\r\nГлава администрации ЗАТО — сити-менеджер, согласно уставу, назначается на эту должность по контракту, который заключается с мэром. Предыдущий глава Игорь Волков покинул пост 1 февраля. Исполняющим обязанности главы ЗАТО был назначен первый заместитель Волкова Николай Диденко, который до 2012 года был вице-мэром Новосибирска.\r\n\r\nКак сообщил Изместьев, на конкурс был подан только один пакет документов — от замначальника по ЖКХ, транспорту и связи администрации Северска Владимира Родыгина. Его документы

In [43]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
deeppavlov_ner(document)

2021-08-08 12:30:33.896 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [11:22<00:00, 972kB/s] 
2021-08-08 12:41:57.129 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
2021-08-08 12:42:06.133 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/datasets/entity_and_type_detection_rus.pickle to /root/.deeppavlov/models/entity_and_type_detection_rus.pickle
100%|██████████| 2.07M/2.07M [00:02<00:00, 835kB/s]
2021-08-08 12:42:11.37 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/models/ner_cq_rus.tar.gz to /root/.deeppavlov/models/ner_cq_rus.tar.gz
100%|

2021-08-08 12:54:04.74 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ent_and_type_rus/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-08-08 12:54:33.382 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ent_and_type_rus/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ent_and_type_rus/model


[[['\r'],
  ['\n'],
  ['\r'],
  ['\n'],
  ['С'],
  ['е'],
  ['в'],
  ['е'],
  ['р'],
  ['с'],
  ['к'],
  [],
  ['д'],
  ['о'],
  [],
  ['к'],
  ['о'],
  ['н'],
  ['ц'],
  ['а'],
  [],
  ['г'],
  ['о'],
  ['д'],
  ['а'],
  [],
  ['о'],
  ['с'],
  ['т'],
  ['а'],
  ['н'],
  ['е'],
  ['т'],
  ['с'],
  ['я'],
  [],
  ['б'],
  ['е'],
  ['з'],
  [],
  ['с'],
  ['и'],
  ['т'],
  ['и'],
  ['-'],
  ['м'],
  ['е'],
  ['н'],
  ['е'],
  ['д'],
  ['ж'],
  ['е'],
  ['р'],
  ['а'],
  ['\r'],
  ['\n'],
  ['\r'],
  ['\n'],
  ['\r'],
  ['\n'],
  ['Д'],
  ['у'],
  ['м'],
  ['а'],
  [],
  ['С'],
  ['е'],
  ['в'],
  ['е'],
  ['р'],
  ['с'],
  ['к'],
  ['а'],
  [],
  ['п'],
  ['р'],
  ['и'],
  ['з'],
  ['н'],
  ['а'],
  ['л'],
  ['а'],
  [],
  ['н'],
  ['е'],
  ['с'],
  ['о'],
  ['с'],
  ['т'],
  ['о'],
  ['я'],
  ['в'],
  ['ш'],
  ['и'],
  ['м'],
  ['с'],
  ['я'],
  [],
  ['к'],
  ['о'],
  ['н'],
  ['к'],
  ['у'],
  ['р'],
  ['с'],
  [],
  ['н'],
  ['а'],
  [],
  ['п'],
  ['о'],
  ['с'],
  ['т'],
  [],
  [

## Написать свой нер попробовать разные подходы
 1. передаём в сетку токен и его соседей
 2. передаём в сетку только токен

In [2]:
!pip install razdel

In [3]:
from razdel import tokenize

In [13]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        # if "http://" in input:
        #   token='None'
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [18]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [19]:
df_words['tag'].value_counts()

OUT         219001
PER          21182
ORG          13651
LOC           4563
GEOPOLIT      4346
MEDIA         2481
Name: tag, dtype: int64

In [20]:
df_words.shape

(265224, 2)

In [21]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input, Bidirectional,Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

In [22]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [23]:
train_x.apply(len).max(axis=0)

55

In [24]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(2048)
valid_data = valid_data.batch(2048)

In [25]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [26]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 3),
    output_sequence_length=seq_len
    )

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [29]:
t=np.unique(encoder.inverse_transform(valid_y),return_counts=True)[1]
t=t/t.sum()
t  # распределение таргета

array([0.01585075, 0.01757005, 0.00924502, 0.05195608, 0.82677284,
       0.07860525])

In [30]:
model = tf.keras.Sequential([
  vectorize_layer,
  tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(300, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(50, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(6, activation='softmax')
  ])

In [31]:
model.compile(optimizer='adam',         
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
98/98 [==============================] - 6s 25ms/step - loss: 0.7278 - accuracy: 0.8198 - val_loss: 0.4003 - val_accuracy: 0.8277
Epoch 2/3
98/98 [==============================] - 2s 24ms/step - loss: 0.2800 - accuracy: 0.9028 - val_loss: 0.2545 - val_accuracy: 0.9271
Epoch 3/3
98/98 [==============================] - 2s 23ms/step - loss: 0.1600 - accuracy: 0.9494 - val_loss: 0.2335 - val_accuracy: 0.9351


In [32]:
pred=model.predict(valid_data)

In [33]:
list(zip(encoder.classes_,pred.mean(axis=0)/t))

[('GEOPOLIT', 1.0064137770407093),
 ('LOC', 0.9470936289193036),
 ('MEDIA', 0.8778728546200967),
 ('ORG', 1.343452731090637),
 ('OUT', 0.9945523157079739),
 ('PER', 0.8559166679611959)]

In [34]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    # tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6,activation='softmax')
])

In [35]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [37]:
model.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
98/98 [==============================] - 17s 67ms/step - loss: 0.8678 - accuracy: 0.8205 - val_loss: 0.4065 - val_accuracy: 0.8626
Epoch 2/3
98/98 [==============================] - 5s 48ms/step - loss: 0.3236 - accuracy: 0.8890 - val_loss: 0.3002 - val_accuracy: 0.8986
Epoch 3/3
98/98 [==============================] - 4s 44ms/step - loss: 0.2136 - accuracy: 0.9359 - val_loss: 0.2530 - val_accuracy: 0.9360
